In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.signal import find_peaks
import os

# Imports pour scikit-learn (KNN)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer 

# Affichage dans Jupyter
try:
    from IPython.display import display, clear_output
except ImportError:
    display = print 
    clear_output = lambda wait=False: os.system('cls' if os.name == 'nt' else 'clear')

In [2]:
# --- Paramètres de Détection des Coups (identiques à analyse_coups.py) ---
SMOOTHING_WINDOW_SIZE_KNN = 15
ACCEL_Y_PEAK_HEIGHT_KNN = 1.0 
ACCEL_Y_FIRST_PEAK_PROMINENCE_KNN = 0.4
ACCEL_Y_SECOND_PEAK_PROMINENCE_KNN = 0.15
ACCEL_Y_PEAK_DISTANCE_KNN = 4
MAX_GAP_SECONDS_FOR_PAIRS_KNN = 1.0
ROTATION_EXTREMA_MARGIN_SECONDS_KNN = 0.25

ENERGY_ACTIVITY_THRESHOLD = 4  # Seuil sur l'enveloppe d'énergie
MIN_DURATION_SAMPLES_ENERGY = 5 # Durée minimale d'un coup en nombre d'échantillons
MERGE_GAP_SAMPLES_ENERGY = 10    # Gap max (échantillons) pour fusionner deux segments actifs
SAMPLING_INTERVAL_SECONDS = 0.01  # Intervalle d'échantillonnage de l'Arduino (10ms = 0.01s)

# Colonnes de caractéristiques (features) pour le KNN (pour information)
FEATURES_COLUMNS = [
    'Min_AccX_Val', 'Max_AccX_Val', 'AccX_Amplitude', 'AccX_Ord(1=Max>Min)',
    'Min_AccZ_Val', 
    'Max_Roll_Val', 'Min_Roll_Val', 'Roll_Ord(1=Max>Min)',
    'Max_Pitch_Val', 'Min_Pitch_Val', 'Pitch_Ord(1=Max>Min)'
]

In [3]:
#Entrainement du fichier commun
import pandas as pd

# Nom du fichier CSV 
nom_du_fichier = '../joblib/Cristian/dataset_entrainement_knn_cristian.csv'
df_training_features = pd.read_csv(nom_du_fichier)

print(df_training_features.head())
print(df_training_features.tail())

   Min_AccX_Val  Max_AccX_Val  AccX_Amplitude  AccX_Ord(1=Max>Min)  \
0     -0.153747      0.013967        0.167713                    0   
1     -0.523560     -0.304873        0.218687                    0   
2     -0.371640     -0.117013        0.254627                    1   
3     -0.346847     -0.183720        0.163127                    0   
4     -0.079220      0.080980        0.160200                    1   

   Min_AccZ_Val  Max_Roll_Val  Min_Roll_Val  Roll_Ord(1=Max>Min)  \
0      1.055120    236.483333   -230.521667                    0   
1      0.970040    192.775333   -236.679333                    0   
2      0.608720    240.947000   -258.601000                    0   
3      0.643967    215.422667   -237.407333                    0   
4      0.859080    247.587667   -275.795333                    0   

   Max_Pitch_Val  Min_Pitch_Val  Pitch_Ord(1=Max>Min) Type_Coup  
0     225.960000    -181.274333                     0    direct  
1     218.045333    -187.453000       

In [9]:
import pandas as pd
import numpy as np 
import joblib
import os

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier



print("--- 1. Simulation de la création des données d'entraînement ---")

# Définition des colonnes de features (caractéristiques)
FEATURES_COLUMNS = [
    'Min_AccX_Val', 'Max_AccX_Val', 'AccX_Amplitude', 'AccX_Ord(1=Max>Min)',
    'Min_AccZ_Val',
    'Max_Roll_Val', 'Min_Roll_Val', 'Roll_Ord(1=Max>Min)',
    'Max_Pitch_Val', 'Min_Pitch_Val', 'Pitch_Ord(1=Max>Min)'
]

print(f"Le modèle sera entraîné sur {len(FEATURES_COLUMNS)} caractéristiques.")


# Définition des chemins pour la sauvegarde
output_dir = "../joblib/Cristian"
MODEL_PATH_JB = os.path.join(output_dir, "knn_model_gant.joblib")
SCALER_PATH_JB = os.path.join(output_dir, "scaler_gant.joblib")
IMPUTER_PATH_JB = os.path.join(output_dir, "imputer_gant.joblib")

# Créer le dossier de sortie s'il n'existe pas
os.makedirs(output_dir, exist_ok=True)

# Vérification que le DataFrame existe (cette partie est gardée de votre code original)
if 'df_training_features' in locals() and isinstance(df_training_features, pd.DataFrame) and not df_training_features.empty:
    print("\n--- 2. Début de l'entraînement du modèle KNN avec les données labellisées ---")
    
    # =================================================================
    # Diagnostic : On vérifie les valeurs manquantes avant toute action.
    print("\n[DIAGNOSTIC] Comptage des valeurs manquantes AVANT nettoyage :")
    print(df_training_features.isnull().sum())
    
    # Correction : On supprime les lignes où l'étiquette 'Type_Coup' est manquante.
    lignes_avant = len(df_training_features)
    df_training_features.dropna(subset=['Type_Coup'], inplace=True)
    lignes_apres = len(df_training_features)
    
    if lignes_avant > lignes_apres:
        print(f"\n[CORRECTION] {lignes_avant - lignes_apres} ligne(s) ont été supprimées car 'Type_Coup' était manquant.")
        print("Les données sont maintenant prêtes pour l'entraînement.\n")
    # =================================================================
    
    # Séparation des données en features (X) et étiquettes (y)
    X_ref = df_training_features[FEATURES_COLUMNS]
    y_ref = df_training_features['Type_Coup']

    print(f"Nombre d'échantillons pour l'entraînement après nettoyage: {len(X_ref)}")
    print(f"Distribution des classes pour l'entraînement:\n{y_ref.value_counts()}")

    # a. Imputation des valeurs manquantes dans les FEATURES (X_ref)
    IMPUTER = SimpleImputer(strategy='median')
    X_ref_imputed = IMPUTER.fit_transform(X_ref)
    print("\n1. Imputer entraîné (pour remplir les NaN des features).")

    # b. Standardisation des données
    SCALER = StandardScaler()
    X_ref_scaled = SCALER.fit_transform(X_ref_imputed)
    print("2. Scaler entraîné (pour mettre les features à la même échelle).")

    # c. Entraînement du classifieur KNN
    KNN_MODEL = KNeighborsClassifier(n_neighbors=3)
    KNN_MODEL.fit(X_ref_scaled, y_ref) 
    print("3. Modèle KNN entraîné avec succès.")
    print(f"    > Classes apprises par le modèle: {KNN_MODEL.classes_}")

    # --- 3. SAUVEGARDE DES COMPOSANTS ENTRAÎNÉS ---
    print("\n--- 3. Sauvegarde du modèle, du scaler et de l'imputer ---")
    try:
        joblib.dump(KNN_MODEL, MODEL_PATH_JB)
        print(f"Modèle KNN sauvegardé dans '{MODEL_PATH_JB}'")
        
        joblib.dump(SCALER, SCALER_PATH_JB)
        print(f"Scaler sauvegardé dans '{SCALER_PATH_JB}'")
        
        joblib.dump(IMPUTER, IMPUTER_PATH_JB)
        print(f"Imputer sauvegardé dans '{IMPUTER_PATH_JB}'")
        
        print("\n=> Tous les composants ont été sauvegardés avec succès.")
        
    except Exception as e:
        print(f"ERREUR lors de la sauvegarde d'un composant: {e}")

else:
    print("\nLe DataFrame 'df_training_features' n'est pas disponible ou est vide.")
    print("Veuillez charger vos données dans un DataFrame nommé 'df_training_features' avant d'exécuter cette cellule.")

--- 1. Simulation de la création des données d'entraînement ---
Le modèle sera entraîné sur 11 caractéristiques.

--- 2. Début de l'entraînement du modèle KNN avec les données labellisées ---

[DIAGNOSTIC] Comptage des valeurs manquantes AVANT nettoyage :
Min_AccX_Val            0
Max_AccX_Val            0
AccX_Amplitude          0
AccX_Ord(1=Max>Min)     0
Min_AccZ_Val            0
Max_Roll_Val            0
Min_Roll_Val            0
Roll_Ord(1=Max>Min)     0
Max_Pitch_Val           0
Min_Pitch_Val           0
Pitch_Ord(1=Max>Min)    0
Type_Coup               1
dtype: int64

[CORRECTION] 1 ligne(s) ont été supprimées car 'Type_Coup' était manquant.
Les données sont maintenant prêtes pour l'entraînement.

Nombre d'échantillons pour l'entraînement après nettoyage: 113
Distribution des classes pour l'entraînement:
Type_Coup
crochet     42
uppercut    40
direct      31
Name: count, dtype: int64

1. Imputer entraîné (pour remplir les NaN des features).
2. Scaler entraîné (pour mettre les fe